# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [7]:
# importando o selenium: import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
path = "/home/francielleandro/Projetos/chromedriver_linux64/chromedriver" #diretorio onde o webdriver esta
navegador = webdriver.Chrome(path) #instanciando o webdriver
#acessando o google e pesquisar a cotação do dolar
#para acessar uma pagina web -> navegador.get("www.google.com") - neste caso abrimos o google
#PEGAR COTAÇÕES DO GOOGLE
def pegar_cotacao_by_google(moeda):
    #BLOCO 1 - Fazendo a pesquisa
    navegador.get("http://www.google.com")
    XPATH = "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input"#xpath obtido atraves da inspeção do console F12
    campo_de_busca = navegador.find_element(By.XPATH,XPATH)#acessar o elemento atraves do XPATH
    campo_de_busca.send_keys(f'Cotação {moeda}') #enviando input para o navegador
    campo_de_busca.send_keys(Keys.ENTER) #enviando o comando enter
    #FIM BLOCO 1
    #navegador.get("https://www.google.com/search?q=cota%C3%A7%C3%A3o+dolar") #O BLOCO 1 pode ser substituido por essa linha
    #BLOCO 2 - Pegando os valores
    XPATH = '//*[@id="knowledge-currency__updatable-data-column"]/div[3]/table/tbody/tr[3]/td[1]/input'
    campo_valor = navegador.find_element(By.XPATH,XPATH)
    valor = campo_valor.get_attribute('value')
    valor_formatado = valor.replace(",",".")
    print(valor_formatado)
    #FIM BLOCO 2 
    return valor_formatado


def pegar_cotacao_ouro():
    navegador.get('https://www.melhorcambio.com/ouro-hoje')
    XPATH = '//*[@id="comercial"]'
    campo_valor = navegador.find_element(By.XPATH,XPATH)#acessar o elemento atraves do XPATH
    valor = campo_valor.get_attribute('value')
    valor_formatado = valor.replace(",",".")
    print(valor_formatado)
    return valor_formatado

cotacao_euro = pegar_cotacao_by_google('euro')
cotacao_dolar = pegar_cotacao_by_google('dolar')
cotacao_ouro = pegar_cotacao_ouro()

navegador.quit()

/tmp/ipykernel_21625/657993768.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome(path) #instanciando o webdriver


6.33
5.53
324.80


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
#importando o pandas para tratar dados
import pandas as pd
PATH = '/home/francielleandro/Projetos/Python/Intensivão python/Aula 3/Produtos.xlsx'
tabela = pd.read_excel(PATH)#importando arquivos

display(tabela)#exibindo a tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando o valor da cotação

In [3]:
tabela.loc[tabela['Moeda'] == 'Dólar','Cotação'] = float(cotacao_dolar)#tabela.loc[0,'Cotação'] = float(cotacao_dolar) ->pode-se utilizar o indice
tabela.loc[tabela['Moeda'] == 'Euro','Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro','Cotação'] = float(cotacao_ouro)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.53,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.33,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.53,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.53,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.33,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.53,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,324.80,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [4]:
tabela['Preço de Compra'] = tabela['Preço Original']*tabela['Cotação']
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.53,5529.9447,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.33,28485.0000,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.53,4976.9447,1.70,7649.915
3,IPhone,799.00,Dólar,5.53,4418.4700,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.33,18990.0000,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.53,2657.0544,2.00,4804.800
6,Joia 20g,20.00,Ouro,324.80,6496.0000,1.15,8050.000


In [5]:
tabela['Preço de Venda'] = tabela['Preço de Compra']*tabela['Margem']
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.53,5529.9447,1.40,7741.92258
1,Carro Renault,4500.00,Euro,6.33,28485.0000,2.00,56970.00000
2,Notebook Dell,899.99,Dólar,5.53,4976.9447,1.70,8460.80599
3,IPhone,799.00,Dólar,5.53,4418.4700,1.70,7511.39900
4,Carro Fiat,3000.00,Euro,6.33,18990.0000,1.90,36081.00000
5,Celular Xiaomi,480.48,Dólar,5.53,2657.0544,2.00,5314.10880
6,Joia 20g,20.00,Ouro,324.80,6496.0000,1.15,7470.40000


### Agora vamos exportar a nova base de preços atualizada

In [6]:
from datetime import date

data_atual = date.today()
print(data_atual)
tabela.to_excel(f'Produtos {data_atual}.xlsx',index= False)

2022-01-12
